In [ ]:
import os
import numpy as np  # For this example, we use numpy for simplicity
os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
#from keras import ops
import numpy as np
import tensorflow as tf
from keras import layers
#from datasets import load_dataset
from collections import Counter
# from conlleval import evaluate

In [ ]:
def read_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sentences, tags = [], []
        sentence, tag = [], []
        for line in file:
            line = line.strip()
            if line:
                word, label = line.rsplit(' ', 1)
                sentence.append(word)
                tag.append(label)
            else:
                if sentence:
                    sentences.append(sentence)
                    tags.append(tag)
                sentence, tag = [], []
        if sentence:  # for the last sentence if there's no newline at the end
            sentences.append(sentence)
            tags.append(tag)
    return sentences, tags

# Reading the data
training_set_path = 'C:/Users/TESTUSER/Desktop/UniversityCoursesFiles/uniYear4/AI/ArabicNamedEntityRecognition/code/data/train_cleaned.txt'
validation_set_path = 'C:/Users/TESTUSER/Desktop/UniversityCoursesFiles/uniYear4/AI/ArabicNamedEntityRecognition/code/data/val_cleaned.txt'

train_sentences, train_tags = read_data(training_set_path)
val_sentences, val_tags = read_data(validation_set_path)

In [ ]:
all_tokens = [token for sentence in train_sentences + val_sentences for token in sentence]
counter = Counter(all_tokens)
vocab_size = 44561  # Adjust if needed based on your dataset size and memory constraints
vocabulary = [token for token, count in counter.most_common(vocab_size - 2)]  # -2 for [UNK] and [PAD]

lookup_layer = keras.layers.StringLookup(vocabulary=vocabulary)
tag_lookup = keras.layers.StringLookup(vocabulary=list(mapping.values()), num_oov_indices=0, mask_token=None)

In [ ]:
def process_data(sentences, tags):
    token_ids = []
    tag_ids = []
    for sentence, tag_seq in zip(sentences, tags):
        token_ids.append(lowercase_and_convert_to_ids(sentence))
        tag_ids.append(tag_lookup(tag_seq))
    return token_ids, tag_ids

train_token_ids, train_tag_ids = process_data(train_sentences, train_tags)
val_token_ids, val_tag_ids = process_data(val_sentences, val_tags)

In [ ]:
def make_dataset(token_ids, tag_ids, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((token_ids, tag_ids))
    dataset = dataset.padded_batch(batch_size, padded_shapes=([None], [None]))
    return dataset.prefetch(tf.data.AUTOTUNE)

batch_size = 32
train_dataset = make_dataset(train_token_ids, train_tag_ids, batch_size)
val_dataset = make_dataset(val_token_ids, val_tag_ids, batch_size)

In [ ]:
def tokenize_and_convert_to_ids(text):
    tokens = text.split()  # For Arabic, you might want a more sophisticated tokenization
    return lowercase_and_convert_to_ids(tf.constant(tokens))

# Sample inference using the trained model
sample_text = "صورة من فئة 500 مليون خلال فترة الانتداب البريطاني على فلسطين."
sample_input = tokenize_and_convert_to_ids(sample_text)
sample_input = ops.reshape(sample_input, shape=[1, -1])
output = ner_model.predict(sample_input)
prediction = np.argmax(output, axis=-1)[0]
prediction = [mapping[i] for i in prediction]
print(prediction)